In [1]:
import os 
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.engine.url import URL
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
import boto3
from scipy.stats import pearsonr
from scipy.stats import gmean
import statsmodels.api as sm

In [2]:
import sys
sys.path.append("/Users/sebastianbedoyamazo/Documents/siwa_git/iluma-kb")
import utils_schema as us
from utils_schema import get_otu_and_taxo
from utils_schema import get_mapfile

In [3]:
us

<module 'utils_schema' from '/Users/sebastianbedoyamazo/Documents/siwa_git/iluma-kb/utils_schema.py'>

In [4]:
load_dotenv("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others/.env", override=True)

True

In [5]:
os.environ
sys.path.append("/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others")
import utils_dataexp as u
u

Reading aws keys from .env successfully!!


<module 'utils_dataexp' from '/Users/sebastianbedoyamazo/Documents/Data-exploration-and-others/utils_dataexp.py'>

In [6]:
s3_client = boto3.client("s3")
s3_resource = boto3.resource("s3")

In [7]:
url = URL.create(
    drivername='redshift+redshift_connector',
    host='iluma-kb-1.cn4ff1ztoyt9.us-east-1.redshift.amazonaws.com',
    port=5439,
    database='dev',
    username='awsuser',
    password='Ilumasiwa1'
)

engine = create_engine(url)
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

redshift_metadata = sa.MetaData(bind=session.bind)

# df performance 

In [36]:
df = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/performance_E267.csv")
df


,Projectid,House,Block,Pen,Trt,BW14,BW28,BW42,BWG8_14,BWG22_28,BWG36_42,BWG0_14,BWG0_28,BWG0_42,FCR8_14,FC22_28,FCR36_42,FCR0_14,FCR0_28,FCR0_42
0,E267,1,1,1,5,446.666667,1364.074074,2857.307692,252.630952,467.777778,719.230769,401.352381,1318.759788,2811.993407,1.478724,2.201900,1.618182,1.283617,1.738675,1.652339
1,E267,1,1,2,8,443.703704,1440.370370,2941.600000,254.489418,545.185185,757.600000,398.760847,1395.427513,2896.657143,1.400563,1.851902,1.572334,1.252232,1.583742,1.569614
2,E267,1,1,3,4,438.518519,1405.555556,2895.384615,248.947090,526.666667,762.307692,395.089947,1362.126984,2851.956044,1.385837,1.997187,1.544904,1.240824,1.652078,1.631945
3,E267,1,1,4,1,455.555556,1479.230769,2985.000000,255.805556,540.769231,740.000000,411.070707,1434.745921,2940.515152,1.412904,1.960882,1.632883,1.230172,1.592463,1.585980
4,E267,1,1,5,6,448.148148,1449.629630,2900.769231,258.683862,518.888889,704.230769,406.091005,1407.572487,2858.712088,1.330913,1.899358,1.685418,1.193008,1.558927,1.617675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,E267,4,2,36,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
156,E267,4,2,37,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
157,E267,4,2,38,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
158,E267,4,2,39,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
#df.rename(columns = {'House':'house', 'Pen':'pen', 'Projectid':'projectid'}, inplace = True)
#df_pilot['house'] = df_pilot['house'].replace('[^0-9\.]', '', regex=True)
df['House'] = df['House'].astype(float).astype(int)
df['Pen'] = df['Pen'].astype(float).astype(int)
df = df.rename(columns={'Projectid': 'ProjectID'})
df.head()

,ProjectID,House,Block,Pen,Trt,BW14,BW28,BW42,BWG8_14,BWG22_28,BWG36_42,BWG0_14,BWG0_28,BWG0_42,FCR8_14,FC22_28,FCR36_42,FCR0_14,FCR0_28,FCR0_42
0,E267,1,1,1,5,446.666667,1364.074074,2857.307692,252.630952,467.777778,719.230769,401.352381,1318.759788,2811.993407,1.478724,2.201900,1.618182,1.283617,1.738675,1.652339
1,E267,1,1,2,8,443.703704,1440.370370,2941.600000,254.489418,545.185185,757.600000,398.760847,1395.427513,2896.657143,1.400563,1.851902,1.572334,1.252232,1.583742,1.569614
2,E267,1,1,3,4,438.518519,1405.555556,2895.384615,248.947090,526.666667,762.307692,395.089947,1362.126984,2851.956044,1.385837,1.997187,1.544904,1.240824,1.652078,1.631945
3,E267,1,1,4,1,455.555556,1479.230769,2985.000000,255.805556,540.769231,740.000000,411.070707,1434.745921,2940.515152,1.412904,1.960882,1.632883,1.230172,1.592463,1.585980
4,E267,1,1,5,6,448.148148,1449.629630,2900.769231,258.683862,518.888889,704.230769,406.091005,1407.572487,2858.712088,1.330913,1.899358,1.685418,1.193008,1.558927,1.617675


# Para long format

In [85]:
df1 = df[['ProjectID', 'House', 'Block', 'Pen', 'Trt', 'BW14', 'BW28', 'BW42']]
df2 = df[['ProjectID', 'House', 'Block', 'Pen', 'Trt', 'BWG8_14', 'BWG22_28', 'BWG36_42']]
df3 = df[['ProjectID', 'House', 'Block', 'Pen', 'Trt', 'BWG0_14', 'BWG0_28', 'BWG0_42']]
df4 = df[['ProjectID', 'House', 'Block', 'Pen', 'Trt', 'FCR8_14', 'FC22_28', 'FCR36_42']]
df5 = df[['ProjectID', 'House', 'Block', 'Pen', 'Trt', 'FCR0_14', 'FCR0_28', 'FCR0_42']]


In [86]:
df_long1 = pd.concat([df.melt(id_vars=['ProjectID', 'House', 'Block', 'Pen', 'Trt'], value_vars=['BW14', 'BW28', 'BW42'], var_name='var', value_name='BW')
])
df_long2 = pd.concat([df.melt(id_vars=['ProjectID', 'House', 'Block', 'Pen', 'Trt'], value_vars=['BWG8_14', 'BWG22_28', 'BWG36_42'], var_name='var', value_name='BWGbefore')
])
df_long3 = pd.concat([df.melt(id_vars=['ProjectID', 'House', 'Block', 'Pen', 'Trt'], value_vars=['BWG0_14', 'BWG0_28', 'BWG0_42'], var_name='var', value_name='BWG')
])
df_long4 = pd.concat([df.melt(id_vars=['ProjectID', 'House', 'Block', 'Pen', 'Trt'], value_vars=['FCR8_14', 'FC22_28', 'FCR36_42'], var_name='var', value_name='FCRbefore')
])
df_long5 = pd.concat([df.melt(id_vars=['ProjectID', 'House', 'Block', 'Pen', 'Trt'], value_vars=['FCR0_14', 'FCR0_28', 'FCR0_42'], var_name='var', value_name='FCR')
])



In [87]:
df_long2 = df_long2[['BWGbefore']]
df_long3 = df_long3[['BWG']]
df_long4 = df_long4[['FCRbefore']]
df_long5 = df_long5[['FCR']]


In [88]:
df_long = pd.concat([df_long1, df_long2, df_long3, df_long4, df_long5], axis=1)
df_long.head()

,ProjectID,House,Block,Pen,Trt,var,BW,BWGbefore,BWG,FCRbefore,FCR
0,E267,1,1,1,5,BW14,446.666667,252.630952,401.352381,1.478724,1.283617
1,E267,1,1,2,8,BW14,443.703704,254.489418,398.760847,1.400563,1.252232
2,E267,1,1,3,4,BW14,438.518519,248.947090,395.089947,1.385837,1.240824
3,E267,1,1,4,1,BW14,455.555556,255.805556,411.070707,1.412904,1.230172
4,E267,1,1,5,6,BW14,448.148148,258.683862,406.091005,1.330913,1.193008


In [89]:
df_long['Age'] = df_long['var'].str.extract('(\d+)', expand=False)
df_long['Age'] = pd.to_numeric(df_long['Age'])
df_long

,ProjectID,House,Block,Pen,Trt,var,BW,BWGbefore,BWG,FCRbefore,FCR,Age
0,E267,1,1,1,5,BW14,446.666667,252.630952,401.352381,1.478724,1.283617,14
1,E267,1,1,2,8,BW14,443.703704,254.489418,398.760847,1.400563,1.252232,14
2,E267,1,1,3,4,BW14,438.518519,248.947090,395.089947,1.385837,1.240824,14
3,E267,1,1,4,1,BW14,455.555556,255.805556,411.070707,1.412904,1.230172,14
4,E267,1,1,5,6,BW14,448.148148,258.683862,406.091005,1.330913,1.193008,14
...,...,...,...,...,...,...,...,...,...,...,...,...
475,E267,4,2,36,0,BW42,0.000000,0.000000,0.000000,NaN,NaN,42
476,E267,4,2,37,0,BW42,0.000000,0.000000,0.000000,NaN,NaN,42
477,E267,4,2,38,0,BW42,0.000000,0.000000,0.000000,NaN,NaN,42
478,E267,4,2,39,0,BW42,0.000000,0.000000,0.000000,NaN,NaN,42


In [90]:
del df_long['var']

In [91]:
df_long.head()

,ProjectID,House,Block,Pen,Trt,BW,BWGbefore,BWG,FCRbefore,FCR,Age
0,E267,1,1,1,5,446.666667,252.630952,401.352381,1.478724,1.283617,14
1,E267,1,1,2,8,443.703704,254.489418,398.760847,1.400563,1.252232,14
2,E267,1,1,3,4,438.518519,248.947090,395.089947,1.385837,1.240824,14
3,E267,1,1,4,1,455.555556,255.805556,411.070707,1.412904,1.230172,14
4,E267,1,1,5,6,448.148148,258.683862,406.091005,1.330913,1.193008,14


# Renombrar columnas si no es necesario aplicar long format

In [4]:
df[[i for i in df.columns if "33" in i]].columns

Index([], dtype='object')

In [5]:
df.columns

Index(['ProjectID', 'House', 'Block', 'Pen', 'Trt', 'FCR', 'FCRbefore'], dtype='object')

In [55]:
df33 = df[['ProjectID', 'House', 'Block', 'Pen', 'Trt','BW0_33', 'BWG0_33', 'FCR0_33']]
df33.rename(columns = {'BW0_33':'BW', 'BWG0_33':'BWG','FCR0_33':'FCR',}, inplace = True)
df33.head()

,ProjectID,House,Block,Pen,Trt,BW,BWG,FCR
0,E335,1,1,1,5,2002.400000,1956.211667,1.495186
1,E335,1,1,2,7,1744.166667,1707.033333,1.787848
2,E335,1,1,3,12,2006.400000,1954.161667,1.475253
3,E335,1,1,4,11,1977.200000,1936.553333,1.408206
4,E335,1,1,5,9,1972.400000,1928.296667,1.486632


In [13]:
df21[['House', 'Pen', 'Trt']].value_counts()

House  Pen  Trt
5      1    2      1
       38   8      1
       28   8      1
       29   10     1
       30   7      1
       31   2      1
       32   10     1
       33   5      1
       34   4      1
       35   1      1
       36   3      1
       37   6      1
       39   7      1
       2    2      1
       40   9      1
       41   8      1
       42   2      1
       43   3      1
       44   1      1
       45   5      1
       46   7      1
       47   6      1
       48   4      1
       49   10     1
       27   1      1
       26   1      1
       25   9      1
       24   4      1
       3    1      1
       4    5      1
       5    3      1
       6    10     1
       7    7      1
       8    9      1
       9    8      1
       10   4      1
       11   2      1
       12   8      1
       13   9      1
       14   6      1
       15   3      1
       16   4      1
       17   10     1
       18   7      1
       19   5      1
       20   6      1
       21   5     

# metadata (projectid, house, pen)

In [8]:
#df_micro = get_mapfile("Fieldale")

In [92]:
query = "SELECT * FROM microbiome as M LEFT JOIN kit as K on M.kitid = K.kitid LEFT JOIN animal as A on M.animalid = A.animalid WHERE projectid = 'E267';"
metadata = pd.read_sql(query, engine)
print(metadata.shape)
print(metadata.columns)
metadata.head()
#metadata.groupby(['treatment', 'treatmentnumber']).count()

(341, 19)
Index(['sampleid', 'fullsampleid', 'kitid', 'runid', 'animalid',
       'samplelocation', 'alphashannon', 'alphaobserved', 'kitid', 'projectid',
       'age', 'treatment', 'treatmentnumber', 'client', 'animalid',
       'animaltype', 'animalnumber', 'house', 'pen'],
      dtype='object')


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,house,pen
0,H011_05F,H011_05F-M,H011,20181130,H011_05,feces,1.593163,33.0,H011,E267,28,Biostrong,6,Iluma,H011_05,Broiler,05,3.0,22.0
1,H011_05C,H011_05C-M,H011,20181130,H011_05,cecum,4.970696,78.0,H011,E267,28,Biostrong,6,Iluma,H011_05,Broiler,05,3.0,22.0
2,H011_05I,H011_05I-M,H011,20181130,H011_05,ileum,2.422285,36.0,H011,E267,28,Biostrong,6,Iluma,H011_05,Broiler,05,3.0,22.0
3,H008_07F,H008_07F-M,H008,20181130,H008_07,feces,2.896209,57.0,H008,E267,28,Varium,4,Iluma,H008_07,Broiler,07,3.0,33.0
4,H008_07I,H008_07I-M,H008,20181130,H008_07,ileum,3.315672,42.0,H008,E267,28,Varium,4,Iluma,H008_07,Broiler,07,3.0,33.0


In [32]:
#metadata.groupby(['pen', 'house', 'treatmentnumber']).count()
#metadata['Nombre'].apply(lambda x: x[:2])
#metadata[['pen', 'house', 'treatmentnumber']].value_counts()

In [93]:
metadata['pen'] = metadata['pen'].astype(float).astype(int)
metadata['house'] = metadata['house'].astype(float).astype(int)
metadata['age'] = metadata['age'].astype(float).astype(int)
metadata['treatmentnumber'] = metadata['treatmentnumber'].astype(float).astype(int)
#metadata['Trt'] = metadata['Trt'].astype(float).astype(int)
metadata['age'] = pd.to_numeric(metadata['age'])
#metadata['treatmentnumber'] = pd.to_numeric(metadata['treatmentnumber'])
metadata = metadata.rename(columns={ 'sampleid': 'SampleID', 'fullsampleid': 'FullSampleID',
                                   'samplelocation': 'SampleLocation','projectid': 'ProjectID','age': 'Age',
                                   'house': 'House','pen': 'Pen'})

In [94]:
metadata = metadata[['SampleID', 'FullSampleID',
       'SampleLocation', 'ProjectID', 'Age', 'House', 'Pen', 'treatmentnumber']]
metadata.head()

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,treatmentnumber
0,H011_05F,H011_05F-M,feces,E267,28,3,22,6
1,H011_05C,H011_05C-M,cecum,E267,28,3,22,6
2,H011_05I,H011_05I-M,ileum,E267,28,3,22,6
3,H008_07F,H008_07F-M,feces,E267,28,3,33,4
4,H008_07I,H008_07I-M,ileum,E267,28,3,33,4


In [95]:
metadata = metadata.rename(columns={'treatmentnumber': 'Trt'})
metadata

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt
0,H011_05F,H011_05F-M,feces,E267,28,3,22,6
1,H011_05C,H011_05C-M,cecum,E267,28,3,22,6
2,H011_05I,H011_05I-M,ileum,E267,28,3,22,6
3,H008_07F,H008_07F-M,feces,E267,28,3,33,4
4,H008_07I,H008_07I-M,ileum,E267,28,3,33,4
...,...,...,...,...,...,...,...,...
336,H015_03I,H015_03I-M,ileum,E267,42,3,40,8
337,H015_03F,H015_03F-M,feces,E267,42,3,40,8
338,H015_03C,H015_03C-M,cecum,E267,42,3,40,8
339,H006_06I,H006_06I-M,ileum,E267,42,2,6,3


# merge metadata with perfomance

In [74]:
df_long.columns

Index(['ProjectID', 'House', 'Block', 'Pen', 'Trt', 'BW', 'BWGbefore', 'BWG',
       'FCRbefore', 'FCR', 'Age'],
      dtype='object')

In [77]:
df_long.head()

,ProjectID,House,Block,Pen,Trt,BW,BWGbefore,BWG,FCRbefore,FCR,Age
0,E267,1,1,1,5,446.666667,252.630952,401.352381,1.478724,1.283617,14
1,E267,1,1,2,8,443.703704,254.489418,398.760847,1.400563,1.252232,14
2,E267,1,1,3,4,438.518519,248.947090,395.089947,1.385837,1.240824,14
3,E267,1,1,4,1,455.555556,255.805556,411.070707,1.412904,1.230172,14
4,E267,1,1,5,6,448.148148,258.683862,406.091005,1.330913,1.193008,14


In [75]:
metadata.columns

Index(['SampleID', 'FullSampleID', 'SampleLocation', 'ProjectID', 'Age',
       'House', 'Pen', 'Trt'],
      dtype='object')

In [76]:
metadata.head()

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt
0,H011_05F,H011_05F-M,feces,E267,28,3,22,6
1,H011_05C,H011_05C-M,cecum,E267,28,3,22,6
2,H011_05I,H011_05I-M,ileum,E267,28,3,22,6
3,H008_07F,H008_07F-M,feces,E267,28,3,33,4
4,H008_07I,H008_07I-M,ileum,E267,28,3,33,4


In [96]:
metadata_per = pd.merge(metadata, df_long, on =['ProjectID', 'Pen', 'Trt', 'House', 'Age'], how='left')
print(metadata_per.shape)
print(metadata_per.columns)
metadata_per.head(3)

(341, 14)
Index(['SampleID', 'FullSampleID', 'SampleLocation', 'ProjectID', 'Age',
       'House', 'Pen', 'Trt', 'Block', 'BW', 'BWGbefore', 'BWG', 'FCRbefore',
       'FCR'],
      dtype='object')


,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR
0,H011_05F,H011_05F-M,feces,E267,28,3,22,6,2,1483.703704,521.851852,1440.389418,1.854507,1.517143
1,H011_05C,H011_05C-M,cecum,E267,28,3,22,6,2,1483.703704,521.851852,1440.389418,1.854507,1.517143
2,H011_05I,H011_05I-M,ileum,E267,28,3,22,6,2,1483.703704,521.851852,1440.389418,1.854507,1.517143


In [98]:
metadata_per.groupby(['Age', 'SampleLocation'])['BW'].mean()

Age  SampleLocation
14   cecum              454.362600
     feces              456.684249
     ileum              457.061369
28   cecum             1517.022236
     feces             1524.941555
     ileum             1532.924608
42   cecum             2955.090309
     feces             2942.999463
     ileum             2947.560664
Name: BW, dtype: float64

In [66]:
metadata_per.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E335.csv", header=True, index=False)

# Concat metadata

In [10]:
metadata267 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E267.csv")
metadata271 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E271.csv")
metadata321 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E321.csv")
metadata325 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E325.csv")
metadata326 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E326.csv")
metadata335 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E335.csv")
metadata345 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E345.csv")
metadata347 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_E347.csv")


In [42]:
metadata267.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR
0,H010_09F,H010_09F-M,feces,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
1,H010_09I,H010_09I-M,ileum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
2,H010_09C,H010_09C-M,cecum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327


In [43]:
metadata271.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR
0,H028_08I,H028_08I-M,ileum,E271,14,4,24,6,2,367.928571,229.214286,327.973661,1.386725,1.390301
1,H028_08C,H028_08C-M,cecum,E271,14,4,24,6,2,367.928571,229.214286,327.973661,1.386725,1.390301
2,H028_08F,H028_08F-M,feces,E271,14,4,24,6,2,367.928571,229.214286,327.973661,1.386725,1.390301


In [44]:
metadata267.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR
0,H010_09F,H010_09F-M,feces,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
1,H010_09I,H010_09I-M,ileum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
2,H010_09C,H010_09C-M,cecum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327


In [45]:
metadata321.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,FCRbefore
0,0015_05F,0015_05F-M,feces,E321,14,5,41,8,5,388.25,NaN,NaN
1,0015_05C,0015_05C-M,cecum,E321,14,5,41,8,5,388.25,NaN,NaN
2,0011_05F,0011_05F-M,feces,E321,14,5,43,3,5,394.00,NaN,NaN


In [47]:
metadata325.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,FCRbefore
0,0032_02I,0032_02I-M,ileum,E325,28,1,17,1,2,1520.714286,556.071429,1.663455
1,0032_02F,0032_02F-M,feces,E325,28,1,17,1,2,1520.714286,556.071429,1.663455
2,0032_02C,0032_02C-M,cecum,E325,28,1,17,1,2,1520.714286,556.071429,1.663455


In [48]:
metadata326.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,Pen,Trt,House,Block,BW,BWGbefore,FCRbefore
0,0041_05C,0041_05C-M,cecum,E326,21,46,7,5,3,854.166667,410.0,1.377846
1,0041_05I,0041_05I-M,ileum,E326,21,46,7,5,3,854.166667,410.0,1.377846
2,0041_05F,0041_05F-M,feces,E326,21,46,7,5,3,854.166667,410.0,1.377846


In [68]:
metadata335.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWG,FCR
0,0073_01I,0073_01I-M,ileum,E335,42,1,14,4,1,2097.2,2046.661667,1.454515
1,0073_01C,0073_01C-M,cecum,E335,42,1,14,4,1,2097.2,2046.661667,1.454515
2,0073_01F,0073_01F-M,feces,E335,42,1,14,4,1,2097.2,2046.661667,1.454515


In [49]:
metadata345.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR
0,0113_05I,0113_05I-M,ileum,E345,42,2,26,9,7,2687.826087,634.347826,2648.387826,1.864976,1.667824
1,0113_05F,0113_05F-M,feces,E345,42,2,26,9,7,2687.826087,634.347826,2648.387826,1.864976,1.667824
2,0113_05C,0113_05C-M,cecum,E345,42,2,26,9,7,2687.826087,634.347826,2648.387826,1.864976,1.667824


In [69]:
metadata347.head(3)

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore
0,0116_03F,0116_03F-M,feces,E347,42,2,18,2,6,2711.0,778.0,2670.65,1.570694
1,0116_03I,0116_03I-M,ileum,E347,42,2,18,2,6,2711.0,778.0,2670.65,1.570694
2,0116_03C,0116_03C-M,cecum,E347,42,2,18,2,6,2711.0,778.0,2670.65,1.570694


In [11]:
metadata_all = pd.concat([metadata267, metadata271, metadata321, metadata325, metadata326,
                         metadata335, metadata345, metadata347], axis=0)
metadata_all

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR
0,H010_09F,H010_09F-M,feces,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
1,H010_09I,H010_09I-M,ileum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
2,H010_09C,H010_09C-M,cecum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
3,H010_03I,H010_03I-M,ileum,E267,14,3,37,6,2,454.074074,261.002646,411.274074,1.256145,1.118636
4,H010_03F,H010_03F-M,feces,E267,14,3,37,6,2,454.074074,261.002646,411.274074,1.256145,1.118636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,0118_01F,0118_01F-M,feces,E347,42,1,7,4,1,2703.500000,811.000000,2669.741667,1.553637,NaN
166,0118_01C,0118_01C-M,cecum,E347,42,1,7,4,1,2703.500000,811.000000,2669.741667,1.553637,NaN
167,0120_04I,0120_04I-M,ileum,E347,42,2,32,6,8,2461.500000,544.000000,2421.020000,2.100184,NaN
168,0120_04F,0120_04F-M,feces,E347,42,2,32,6,8,2461.500000,544.000000,2421.020000,2.100184,NaN


In [74]:
metadata_all['SampleID'].value_counts()

H010_09F    1
0081_02C    1
0081_02F    1
0082_05C    1
0082_05F    1
           ..
H027_10C    1
H027_10I    1
H027_10F    1
H027_02F    1
0120_04C    1
Name: SampleID, Length: 1725, dtype: int64

In [9]:
metadata_all.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_broiler.csv", header=True, index=False)

In [8]:
metadata_all = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_broiler.csv")
metadata_all

,SampleID,SampleLocation,ProjectID,Age,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR,Treatment,cat_1_BW,cat_1_BWGbefore,cat_1_BWG,cat_1_FCRbefore,cat_1_FCR,House
0,0008_01C,cecum,E321,14,1,2,1,390.3333,NaN,NaN,NaN,NaN,2,Medium,NaN,NaN,NaN,NaN,NaN
1,0008_01F,feces,E321,14,1,2,1,390.3333,NaN,NaN,NaN,NaN,2,Medium,NaN,NaN,NaN,NaN,NaN
2,0008_02C,cecum,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
3,0008_02F,feces,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
4,0008_02I,ileum,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,0226_12C,C,Fieldale,28,23,B,12,NaN,NaN,NaN,1.466042,NaN,Fractal,NaN,NaN,NaN,Medium,NaN,1.0
2003,0225_01I,I,Fieldale,28,1,A,1,NaN,NaN,NaN,1.546512,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0
2004,0225_02I,I,Fieldale,28,3,A,2,NaN,NaN,NaN,1.483436,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0
2005,0225_03I,I,Fieldale,28,5,A,3,NaN,NaN,NaN,1.521951,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0


In [8]:
#con fieldale
metadata_all = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Joyce and Fieldale/Fieldale/meta_exp_broiler.csv")
metadata_all


,SampleID,SampleLocation,ProjectID,Age,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR,Treatment,cat_1_BW,cat_1_BWGbefore,cat_1_BWG,cat_1_FCRbefore,cat_1_FCR,House
0,0008_01C,cecum,E321,14,1,2,1,390.3333,NaN,NaN,NaN,NaN,2,Medium,NaN,NaN,NaN,NaN,NaN
1,0008_01F,feces,E321,14,1,2,1,390.3333,NaN,NaN,NaN,NaN,2,Medium,NaN,NaN,NaN,NaN,NaN
2,0008_02C,cecum,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
3,0008_02F,feces,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
4,0008_02I,ileum,E321,14,2,2,3,383.8333,NaN,NaN,NaN,NaN,2,Low,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,0226_12C,C,Fieldale,28,23,B,12,NaN,NaN,NaN,1.466042,NaN,Fractal,NaN,NaN,NaN,Medium,NaN,1.0
2003,0225_01I,I,Fieldale,28,1,A,1,NaN,NaN,NaN,1.546512,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0
2004,0225_02I,I,Fieldale,28,3,A,2,NaN,NaN,NaN,1.483436,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0
2005,0225_03I,I,Fieldale,28,5,A,3,NaN,NaN,NaN,1.521951,NaN,Control,NaN,NaN,NaN,Medium,NaN,1.0


In [13]:
len(metadata_all.SampleID.unique())
metadata_all[metadata_all.SampleLocation != "feces"].shape

(1165, 14)

In [4]:
metadata_all.groupby(["Age"]).count()

,SampleID,FullSampleID,SampleLocation,ProjectID,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR
Age,,,,,,,,,,,,,
14,346,346,346,346,346,346,346,346,346,227,227,227,227
21,226,226,226,226,226,226,226,226,226,226,0,226,0
28,327,327,327,327,327,327,327,327,327,327,108,327,108
42,826,826,826,826,826,826,826,826,826,578,826,578,656


# merge with histo and geneexpression

In [7]:
query = "SELECT * FROM histopathology;"
histo = pd.read_sql(query, engine)
print(histo.shape)
print(histo.columns)
histo.head()

(1166, 8)
Index(['sampleid', 'researchnumber', 'additivescore', 'overallarchitecture',
       'mucosalintegrity', 'lymphoidimmune', 'inflammationseverity',
       'microbialorganisms'],
      dtype='object')


,sampleid,researchnumber,additivescore,overallarchitecture,mucosalintegrity,lymphoidimmune,inflammationseverity,microbialorganisms
0,0055_03I,R20-00429,0.0,0.0,0.0,0.0,0.0,0.0
1,0055_04I,R20-00429,3.5,1.0,0.5,0.5,0.5,1.0
2,0055_05I,R20-00429,2.5,0.0,0.5,0.5,0.5,1.0
3,0055_06I,R20-00429,2.5,0.5,0.5,0.5,0.5,0.5
4,0055_08I,R20-00429,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
histo.rename(columns = {'sampleid':'SampleID'}, inplace = True)
histo.head()

,SampleID,researchnumber,additivescore,overallarchitecture,mucosalintegrity,lymphoidimmune,inflammationseverity,microbialorganisms
0,0055_03I,R20-00429,0.0,0.0,0.0,0.0,0.0,0.0
1,0055_04I,R20-00429,3.5,1.0,0.5,0.5,0.5,1.0
2,0055_05I,R20-00429,2.5,0.0,0.5,0.5,0.5,1.0
3,0055_06I,R20-00429,2.5,0.5,0.5,0.5,0.5,0.5
4,0055_08I,R20-00429,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
query = "SELECT * FROM geneexpression;"
geneexp = pd.read_sql(query, engine)
print(geneexp.shape)
print(geneexp.columns)
geneexp.head()

(1166, 5)
Index(['sampleid', 'platecode', 'deltacq_muc2', 'deltacq_il1b',
       'deltacq_il10'],
      dtype='object')


,sampleid,platecode,deltacq_muc2,deltacq_il1b,deltacq_il10
0,0055_01I,20210302_1,-1.888698,10.393615,-0.110909
1,0055_02C,20210302_1,1.975110,9.773539,-0.007244
2,0055_02I,20210302_1,-1.773921,9.406740,0.095983
3,0055_04C,20210302_1,-0.036128,7.464501,-0.253695
4,0056_01C,20210302_1,-0.087632,9.706642,-0.047598


In [9]:
geneexp.rename(columns = {'sampleid':'SampleID'}, inplace = True)
geneexp.head()

,SampleID,platecode,deltacq_muc2,deltacq_il1b,deltacq_il10
0,0055_01I,20210302_1,-1.888698,10.393615,-0.110909
1,0055_02C,20210302_1,1.975110,9.773539,-0.007244
2,0055_02I,20210302_1,-1.773921,9.406740,0.095983
3,0055_04C,20210302_1,-0.036128,7.464501,-0.253695
4,0056_01C,20210302_1,-0.087632,9.706642,-0.047598


In [23]:
len(geneexp.SampleID.unique())

(1166, 5)

In [14]:
metadata_gen = pd.merge(metadata_all, geneexp, on=['SampleID', 'SampleID'], how='left')

print(metadata_gen.shape)
print(metadata_gen.columns)
metadata_gen.head(3)

(1772, 18)
Index(['SampleID', 'FullSampleID', 'SampleLocation', 'ProjectID', 'Age',
       'House', 'Pen', 'Trt', 'Block', 'BW', 'BWGbefore', 'BWG', 'FCRbefore',
       'FCR', 'platecode', 'deltacq_muc2', 'deltacq_il1b', 'deltacq_il10'],
      dtype='object')


,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR,platecode,deltacq_muc2,deltacq_il1b,deltacq_il10
0,H010_09F,H010_09F-M,feces,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327,NaN,NaN,NaN,NaN
1,H010_09I,H010_09I-M,ileum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327,NaN,NaN,NaN,NaN
2,H010_09C,H010_09C-M,cecum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327,NaN,NaN,NaN,NaN


In [21]:
len(metadata_gen.SampleID.unique())
metadata_gen[metadata_gen.duplicated(subset=['SampleID'])]
metadata_gen[metadata_gen.SampleID == "0074_01I"]

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR,platecode,deltacq_muc2,deltacq_il1b,deltacq_il10
1133,0074_01I,0074_01I-M,ileum,E335,42,1,1,5,1,2002.4,NaN,1956.211667,NaN,1.495186,20211213_2,-3.132765,NaN,NaN
1134,0074_01I,0074_01I-M,ileum,E335,42,1,1,5,1,2002.4,NaN,1956.211667,NaN,1.495186,20220204_1,NaN,6.837399,-1.396292


In [22]:
metadata_gen_hist = pd.merge(metadata_gen, histo, on=['SampleID'], how='left')

print(metadata_gen_hist.shape)
print(metadata_gen_hist.columns)
metadata_gen_hist.head(3)

(1772, 25)
Index(['SampleID', 'FullSampleID', 'SampleLocation', 'ProjectID', 'Age',
       'House', 'Pen', 'Trt', 'Block', 'BW', 'BWGbefore', 'BWG', 'FCRbefore',
       'FCR', 'platecode', 'deltacq_muc2', 'deltacq_il1b', 'deltacq_il10',
       'researchnumber', 'additivescore', 'overallarchitecture',
       'mucosalintegrity', 'lymphoidimmune', 'inflammationseverity',
       'microbialorganisms'],
      dtype='object')


,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,...,deltacq_muc2,deltacq_il1b,deltacq_il10,researchnumber,additivescore,overallarchitecture,mucosalintegrity,lymphoidimmune,inflammationseverity,microbialorganisms
0,H010_09F,H010_09F-M,feces,E267,14,1,20,6,1,446.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H010_09I,H010_09I-M,ileum,E267,14,1,20,6,1,446.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H010_09C,H010_09C-M,cecum,E267,14,1,20,6,1,446.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# otu_table and taxonomy for metadata

In [12]:
samples = list(metadata_all['SampleID'])
len(samples)

1725

In [13]:
otu_table, taxonomy = us.get_otus_taxo_fromsamples(samples)

Shape otu table (9413, 1725)
Shape taxonomy (9413, 8)


In [81]:
otu_table.head(3)
otu_table["OTU"] = otu_table.index

In [18]:
otu_table = otu_table.reset_index(drop=False)
otu_table.rename(columns = {'index':'OTU'}, inplace = True)
otu_table.head()

,OTU,0008_01C,0008_01F,0008_02C,0008_02F,0008_02I,0008_03C,0008_03F,0008_03I,0008_04C,...,H029_03I,H029_04I,H029_05C,H029_05I,H029_06I,H029_07C,H029_07F,H029_08C,H029_08F,H029_09C
0,0001d123420b59585627edf5a1292ae8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00046daee9d2d02f078771e881bf0e79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0008a288f7c98ac22b6f496a93ae01e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0011ac4d2a454f4451c263073131dfb3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00176c6f0de3e7f6822cf8fa80411ac5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
otu_table.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/otu_table_broiler.csv", header=True, index=False)


In [20]:
otu_table.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Joyce and Fieldale/Fieldale/otu_table_broiler.csv", header=True, index=False)

In [19]:
taxonomy.rename(columns = {'otu':'OTU', 'species':'Species', 'genus':'Genus', 'family':'Family', 'order':'Order', 'tclass':'Class', 'phylum':'Phylum', 'kingdom':'Kingdom'}, inplace = True)
taxonomy.head()
print(taxonomy.shape)

(9413, 8)


In [85]:
taxonomy.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/taxonomy_broiler.csv", header=True, index=False)

In [21]:
taxonomy.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Joyce and Fieldale/Fieldale/taxonomy_broiler.csv", header=True, index=False)

# Check incosistences

## BUG 347

In [113]:
a = metadata.SampleID
b = metadata_per.SampleID
diff = list(set(a) - set(b))
diff

['0115_05F',
 '0116_05I',
 '0116_05F',
 '0116_05C',
 '0123_04I',
 '0115_05C',
 '0123_04F',
 '0115_05I',
 '0123_04C']

In [114]:
test = metadata[metadata['SampleID'].isin(diff)]
test

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt
12,0115_05F,0115_05F-M,feces,E347,42,3,2,1
13,0115_05I,0115_05I-M,ileum,E347,42,3,2,1
14,0115_05C,0115_05C-M,cecum,E347,42,3,2,1
57,0123_04I,0123_04I-M,ileum,E347,42,2,35,9
58,0123_04F,0123_04F-M,feces,E347,42,2,35,9
59,0123_04C,0123_04C-M,cecum,E347,42,2,35,9
176,0116_05I,0116_05I-M,ileum,E347,42,3,3,2
177,0116_05F,0116_05F-M,feces,E347,42,3,3,2
178,0116_05C,0116_05C-M,cecum,E347,42,3,3,2


In [115]:
test[['House', 'Pen', 'Trt']].value_counts()

House  Pen  Trt
2      35   9      3
3      2    1      3
       3    2      3
dtype: int64

In [90]:
df21[['House', 'Pen', 'Trt']].value_counts()


House  Pen  Trt
1      1    2      1
4      38   8      1
3      28   8      1
       29   10     1
       30   5      1
4      31   2      1
       32   10     1
       33   5      1
       34   4      1
       35   1      1
       36   6      1
       37   3      1
       39   7      1
1      2    6      1
4      40   9      1
5      41   8      1
       42   2      1
       43   3      1
       44   1      1
       45   5      1
       46   7      1
       47   6      1
       48   4      1
       49   10     1
3      27   6      1
       26   1      1
       25   9      1
       24   4      1
1      3    1      1
       4    5      1
       5    3      1
       6    10     1
       7    7      1
       8    9      1
       9    8      1
       10   4      1
2      11   2      1
       12   8      1
       13   9      1
       14   6      1
       15   3      1
       16   4      1
       17   10     1
       18   7      1
       19   5      1
       20   1      1
3      21   7     

In [260]:
pen = test.Pen

In [261]:
df[df['Pen'].isin(pen)]

,ProjectID,House,Block,Pen,Trt,BW42,BWG36_42,BWG0_42,FCR36_42
1,E347,1,1,2,5,2694.210526,766.842105,2641.032105,1.589568
2,E347,1,1,3,10,2506.500000,602.500000,2459.000000,1.950207
34,E347,1,4,35,2,2540.500000,660.500000,2505.330000,1.806207
41,E347,2,5,2,10,2504.000000,571.000000,2459.130000,1.945709
42,E347,2,5,3,1,2448.500000,642.500000,2397.800000,1.813230
74,E347,2,8,35,10,2535.000000,633.500000,2492.400000,1.868982
81,E347,3,9,2,2,2579.000000,685.000000,2539.058333,1.756934
82,E347,3,9,3,1,2449.500000,627.000000,2415.706522,1.895534
114,E347,3,12,35,1,2466.000000,656.500000,2429.570000,1.769992
121,E347,4,13,2,4,2639.500000,647.500000,2600.560000,1.871815


In [241]:
E326[E326['Battery'].isin([5])]

,Battery,Floor,Cage,Blind Trts,Trt,BW12 /A (g),BW14 /A (g),FI 13-14d /A (g),BWG 13-14d /A (g),FCR 13-14d (g:g),...,FCR 22-28d (g:g) corrected,Mortality 22-28d (%),Culling 22-28d (%),BW35 /A (g),FI 29-35d /A (g),BWG 29-35d /A (g),FCR 29-35d (g:g),FCR 29-35d corrected (g:g),Mortality 29-35d (%),Culling 29-35d (%)
40,5,5,41,8,Phylox,359.166667,437.500000,93.416667,78.333333,1.192553,...,1.523985,0,0.000000,2090.000000,986.333333,628.333333,1.569761,1.569761,0,0
41,5,5,42,2,Uncllgd + untrtd,360.000000,435.000000,94.916667,75.000000,1.265556,...,1.715632,0,0.000000,2400.000000,1011.777778,649.333333,1.558179,1.558179,0,0
42,5,4,43,3,Avinova+Ganoderma,360.000000,436.666667,97.416667,76.666667,1.270652,...,1.308215,0,0.000000,2208.888889,1089.777778,716.111111,1.521800,1.521800,0,0
43,5,4,44,1,Chllgd + untrtd,358.333333,435.833333,99.416667,77.500000,1.282796,...,1.424581,0,0.000000,2168.888889,1053.111111,682.111111,1.543900,1.543900,0,0
44,5,3,45,5,eXolution+Celerity,359.166667,440.833333,97.666667,81.666667,1.195918,...,1.327769,0,0.000000,2074.444444,1010.555556,620.000000,1.629928,1.629928,0,0
45,5,3,46,7,Cocci Aid,355.833333,444.166667,102.916667,88.333333,1.165094,...,1.426172,0,0.000000,2157.777778,1043.888889,680.666667,1.533627,1.533627,0,0
46,5,2,47,6,Natur Poultry,360.000000,437.500000,97.750000,77.500000,1.261290,...,1.490268,0,0.000000,2224.444444,881.222222,733.444444,1.201485,1.201485,0,0
47,5,2,48,4,Maxiban,355.000000,434.166667,100.750000,79.166667,1.272632,...,1.482061,0,0.000000,2145.555556,1058.000000,736.222222,1.437066,1.437066,0,0
48,5,1,49,10,CTCzyme,357.500000,433.333333,99.833333,75.833333,1.316484,...,1.486397,0,8.333333,2147.500000,1200.375000,508.125000,2.362362,1.892217,0,0
49,5,1,50,9,Avinova+eXolution,355.000000,430.833333,97.916667,75.833333,1.291209,...,1.558356,0,0.000000,1922.222222,950.555556,522.444444,1.819439,1.819439,0,0


In [195]:
E271[E271['Corral'].isin([15])]

,Galpon,Bloque,Zona,Lado,Corral,Tratamiento,Aves finales sem6,Aves finales sem1,BW8/A Fin. W1,"BW1, G",...,Mort+desc 14a21 (%),Mort+desc 21a28 (%),Mort+desc 28a35 (%),Mort+desc 35a42 (%),Mort+desc 8a21 (%),Mort+desc 21a35 (%),Mort+desc 0a42 (%),"Mort+cull 29-42, %","Mort+cull 22-42, %","Mort+cull 0-42, %"
14,1,2,Culata,A,15,3.0,27,33,156.606061,43.828571,...,3.333333,0.0,6.896552,0.0,#REF!,6.896552,15.944171,6.896552,6.896552,15.944171
54,2,2,Culata,A,15,4.0,25,32,143.40625,41.314286,...,3.703704,0.0,3.846154,0.0,#REF!,3.846154,19.246286,3.846154,3.846154,15.542582
94,3,2,Culata,A,15,5.0,25,33,143.787879,40.085714,...,3.703704,0.0,3.846154,0.0,#REF!,3.846154,13.264143,3.846154,3.846154,13.264143
134,4,2,Culata,A,15,6.0,26,32,129.84375,39.542857,...,3.571429,0.0,3.703704,0.0,#REF!,3.703704,18.971561,3.703704,3.703704,18.848408


In [239]:
E271 = pd.read_excel("/Users/sebastianbedoyamazo/Library/CloudStorage/GoogleDrive-sebastian.bedoya@premexcorp.com/.shortcut-targets-by-id/15mKBa5ogkSaPz2wAQGRuDxS32hSeoaBw/Analisis/E271/performance_data/Libro de datos E-271 Probióticos NF revisión 18-07-2022.xlsx", "Datos Estadística")
dfE271 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/performance_E326.csv")
E326 = pd.read_excel("/Users/sebastianbedoyamazo/Documents/experiments/Raw Data E-326 2nd Cocci Trial .xlsx")
dfE326 = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/performance_E326.csv")

# 326

In [25]:
metadata = pd.concat([metadata1, metadata2], axis=0)
metadata

,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved,kitid,projectid,age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,house,pen
0,0015_05F,0015_05F-M,0015,20210527,0015_05,feces,4.059460,117.0,0015,E321,14,AVINOVA+COCCIAID,2,Iluma,0015_05,Broiler,05,5.0,41.0
1,0015_05C,0015_05C-M,0015,20210527,0015_05,cecum,6.139127,184.0,0015,E321,14,AVINOVA+COCCIAID,2,Iluma,0015_05,Broiler,05,5.0,41.0
2,0011_05F,0011_05F-M,0011,20210527,0011_05,feces,1.894555,74.0,0011,E321,14,MONECYL,6,Iluma,0011_05,Broiler,05,5.0,43.0
3,0011_05C,0011_05C-M,0011,20210527,0011_05,cecum,6.318653,178.0,0011,E321,14,MONECYL,6,Iluma,0011_05,Broiler,05,5.0,43.0
4,0022_02F,0022_02F-M,0022,20210527,0022_02,feces,1.472269,23.0,0022,E321,21,SALINOMIX+COCCIAID,9,Iluma,0022_02,Broiler,02,5.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0009_03F,0009_03F-M,0009,20210527,0009_03,feces,4.783254,140.0,0009,E321-2,14.0,Chllgd+Untrtd,4,Iluma,0009_03,Broiler,03,5.0,27.0
209,0009_03C,0009_03C-M,0009,20210527,0009_03,cecum,5.920473,167.0,0009,E321-2,14.0,Chllgd+Untrtd,4,Iluma,0009_03,Broiler,03,5.0,27.0
210,0021_04I,0021_04I-M,0021,20211206,0021_04,ileum,2.752442,18.0,0021,E321-2,21.0,MONECYL,6,Iluma,0021_04,Broiler,04,5.0,36.0
211,0021_04F,0021_04F-M,0021,20210527,0021_04,feces,3.218315,48.0,0021,E321-2,21.0,MONECYL,6,Iluma,0021_04,Broiler,04,5.0,36.0


In [27]:
metadata

,SampleID,FullSampleID,kitid,runid,animalid,SampleLocation,alphashannon,alphaobserved,kitid,ProjectID,Age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,House,Pen
0,0015_05F,0015_05F-M,0015,20210527,0015_05,feces,4.059460,117.0,0015,E321,14,AVINOVA+COCCIAID,2,Iluma,0015_05,Broiler,05,5,41
1,0015_05C,0015_05C-M,0015,20210527,0015_05,cecum,6.139127,184.0,0015,E321,14,AVINOVA+COCCIAID,2,Iluma,0015_05,Broiler,05,5,41
2,0011_05F,0011_05F-M,0011,20210527,0011_05,feces,1.894555,74.0,0011,E321,14,MONECYL,6,Iluma,0011_05,Broiler,05,5,43
3,0011_05C,0011_05C-M,0011,20210527,0011_05,cecum,6.318653,178.0,0011,E321,14,MONECYL,6,Iluma,0011_05,Broiler,05,5,43
4,0022_02F,0022_02F-M,0022,20210527,0022_02,feces,1.472269,23.0,0022,E321,21,SALINOMIX+COCCIAID,9,Iluma,0022_02,Broiler,02,5,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0009_03F,0009_03F-M,0009,20210527,0009_03,feces,4.783254,140.0,0009,E321-2,14,Chllgd+Untrtd,4,Iluma,0009_03,Broiler,03,5,27
209,0009_03C,0009_03C-M,0009,20210527,0009_03,cecum,5.920473,167.0,0009,E321-2,14,Chllgd+Untrtd,4,Iluma,0009_03,Broiler,03,5,27
210,0021_04I,0021_04I-M,0021,20211206,0021_04,ileum,2.752442,18.0,0021,E321-2,21,MONECYL,6,Iluma,0021_04,Broiler,04,5,36
211,0021_04F,0021_04F-M,0021,20210527,0021_04,feces,3.218315,48.0,0021,E321-2,21,MONECYL,6,Iluma,0021_04,Broiler,04,5,36


In [28]:
metadata['Trt'] = metadata['treatment'].apply(lambda x: {'Chllgd+Untrtd': 1, 
                                                    'Unchllgd+Untrt': 2,
                                                    'MONECYL': 3,
                                                    'SALINOMIX': 4,
                                                    'AVINOVA': 5,
                                                    'MAXIBAN': 6,
                                                    'COCCIAID': 7,
                                                    'AVINOVA+COCCIAID': 8,
                                                    'PHYLOX': 9,
                                                    'SALINOMIX+COCCIAID': 10}.get(x, x))





In [30]:
metadata['ProjectID'] = metadata['ProjectID'].apply(lambda x: {'E321-2': 'E321'}.get(x, x))
metadata

,SampleID,FullSampleID,kitid,runid,animalid,SampleLocation,alphashannon,alphaobserved,kitid,ProjectID,Age,treatment,treatmentnumber,client,animalid,animaltype,animalnumber,House,Pen,Trt
0,0015_05F,0015_05F-M,0015,20210527,0015_05,feces,4.059460,117.0,0015,E321,14,AVINOVA+COCCIAID,2,Iluma,0015_05,Broiler,05,5,41,8
1,0015_05C,0015_05C-M,0015,20210527,0015_05,cecum,6.139127,184.0,0015,E321,14,AVINOVA+COCCIAID,2,Iluma,0015_05,Broiler,05,5,41,8
2,0011_05F,0011_05F-M,0011,20210527,0011_05,feces,1.894555,74.0,0011,E321,14,MONECYL,6,Iluma,0011_05,Broiler,05,5,43,3
3,0011_05C,0011_05C-M,0011,20210527,0011_05,cecum,6.318653,178.0,0011,E321,14,MONECYL,6,Iluma,0011_05,Broiler,05,5,43,3
4,0022_02F,0022_02F-M,0022,20210527,0022_02,feces,1.472269,23.0,0022,E321,21,SALINOMIX+COCCIAID,9,Iluma,0022_02,Broiler,02,5,17,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,0009_03F,0009_03F-M,0009,20210527,0009_03,feces,4.783254,140.0,0009,E321,14,Chllgd+Untrtd,4,Iluma,0009_03,Broiler,03,5,27,1
209,0009_03C,0009_03C-M,0009,20210527,0009_03,cecum,5.920473,167.0,0009,E321,14,Chllgd+Untrtd,4,Iluma,0009_03,Broiler,03,5,27,1
210,0021_04I,0021_04I-M,0021,20211206,0021_04,ileum,2.752442,18.0,0021,E321,21,MONECYL,6,Iluma,0021_04,Broiler,04,5,36,3
211,0021_04F,0021_04F-M,0021,20210527,0021_04,feces,3.218315,48.0,0021,E321,21,MONECYL,6,Iluma,0021_04,Broiler,04,5,36,3


# llamar alpha diversity

In [14]:
metadata_all = pd.read_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/metadata_broiler.csv")
metadata_all

,SampleID,FullSampleID,SampleLocation,ProjectID,Age,House,Pen,Trt,Block,BW,BWGbefore,BWG,FCRbefore,FCR
0,H010_09F,H010_09F-M,feces,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
1,H010_09I,H010_09I-M,ileum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
2,H010_09C,H010_09C-M,cecum,E267,14,1,20,6,1,446.666667,261.309524,404.266667,1.374943,1.203327
3,H010_03I,H010_03I-M,ileum,E267,14,3,37,6,2,454.074074,261.002646,411.274074,1.256145,1.118636
4,H010_03F,H010_03F-M,feces,E267,14,3,37,6,2,454.074074,261.002646,411.274074,1.256145,1.118636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720,0118_01F,0118_01F-M,feces,E347,42,1,7,4,1,2703.500000,811.000000,2669.741667,1.553637,NaN
1721,0118_01C,0118_01C-M,cecum,E347,42,1,7,4,1,2703.500000,811.000000,2669.741667,1.553637,NaN
1722,0120_04I,0120_04I-M,ileum,E347,42,2,32,6,8,2461.500000,544.000000,2421.020000,2.100184,NaN
1723,0120_04F,0120_04F-M,feces,E347,42,2,32,6,8,2461.500000,544.000000,2421.020000,2.100184,NaN


In [13]:
query = "SELECT * FROM microbiome;"
metadata = pd.read_sql(query, engine)
print(metadata.shape)
print(metadata.columns)
metadata.head()


(4478, 8)
Index(['sampleid', 'fullsampleid', 'kitid', 'runid', 'animalid',
       'samplelocation', 'alphashannon', 'alphaobserved'],
      dtype='object')


,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved
0,H001_01C,H001_01C-M,H001,20181130,H001_01,cecum,5.775365,114.0
1,H001_02C,H001_02C-M,H001,20181130,H001_02,cecum,5.967654,148.0
2,H001_02F,H001_02F-M,H001,20181130,H001_02,feces,2.558017,26.0
3,H001_03F,H001_03F-M,H001,20181130,H001_03,feces,4.429706,106.0
4,H001_03I,H001_03I-M,H001,20181130,H001_03,ileum,3.153810,43.0


In [15]:
samples = list(metadata_all['SampleID'])
len(samples)

1725

In [17]:
filtered_metadata = metadata[metadata['sampleid'].isin(samples)]
filtered_metadata

,sampleid,fullsampleid,kitid,runid,animalid,samplelocation,alphashannon,alphaobserved
0,H001_01C,H001_01C-M,H001,20181130,H001_01,cecum,5.775365,114.0
1,H001_02C,H001_02C-M,H001,20181130,H001_02,cecum,5.967654,148.0
2,H001_02F,H001_02F-M,H001,20181130,H001_02,feces,2.558017,26.0
3,H001_03F,H001_03F-M,H001,20181130,H001_03,feces,4.429706,106.0
4,H001_03I,H001_03I-M,H001,20181130,H001_03,ileum,3.153810,43.0
...,...,...,...,...,...,...,...,...
4016,0123_01I,0123_01I-M,0123,20220930,0123_01,ileum,2.996650,81.0
4017,0123_02I,0123_02I-M,0123,20220930,0123_02,ileum,2.242916,37.0
4018,0123_05I,0123_05I-M,0123,20220930,0123_05,ileum,2.094877,36.0
4019,0124_03I,0124_03I-M,0124,20220930,0124_03,ileum,2.716216,57.0


In [22]:
filtered_metadata = filtered_metadata.loc[:, ['sampleid', 'alphashannon', 'alphaobserved']]
filtered_metadata

,sampleid,alphashannon,alphaobserved
0,H001_01C,5.775365,114.0
1,H001_02C,5.967654,148.0
2,H001_02F,2.558017,26.0
3,H001_03F,4.429706,106.0
4,H001_03I,3.153810,43.0
...,...,...,...
4016,0123_01I,2.996650,81.0
4017,0123_02I,2.242916,37.0
4018,0123_05I,2.094877,36.0
4019,0124_03I,2.716216,57.0


In [23]:
filtered_metadata.to_csv("/Users/sebastianbedoyamazo/Documents/siwa_git/Performance/performance_df/pq_obj/diversity_broiler.csv", header=True, index=False)